In [3]:
import pyvisa
import time
import matplotlib.pyplot as plt 
import numpy as np
import csv 
import os
from pyvisa.constants import ControlFlow, Parity, StopBits
import pandas as pd

In [4]:
gpib_address = 1
rm = pyvisa.ResourceManager()
bnc = rm.open_resource(f"GPIB::{gpib_address}", )
print(rm.list_opened_resources())

[<'GPIBInstrument'('GPIB0::1::INSTR')>]


In [73]:
#get the bnc box in the correct state
bnc.write_termination = '\r\n'
bnc.read_termination = '\r\n'
bnc.timeout = 5000

In [210]:
#check if termination works
bnc.query("*IDN?")

'555-2-000-01.00'

In [253]:
#always reset
bnc.write("*RST")

6

In [254]:
#set it to be triggered by the 60Hz input from the DS360
bnc.write(":PULSE0:MODE BURST")
bnc.write(":PULSE0:EXT:MODE TRIG")
bnc.write(":PULSE0:EXT:EDGE RIS")
bnc.write(":PULSE0:EXT:LEV 1.0")
bnc.write(":PULSE0:STAT ON")
bnc.write(":PULSE0:PER 0.00032") #system period to 300Hz
bnc.write(":PULSE0:BCO 5") #define the max BCO for other channels also

15

In [255]:
#300 Hz from 60Hz signal
bnc.write(":PULSE1:STAT ON")
bnc.write(":PULSE1:CMOD BURST")
bnc.write(":PULSE1:BCO 5") #changes number of pulses
bnc.write(":PULSE1:WIDT 0.0001")
bnc.write(":PULSE1:DEL 0")


15

In [262]:
#DAQ signal 
bnc.write(":PULSE2:STAT ON")
bnc.write(":PULSE2:CMOD BURST")
bnc.write(":PULSE2:BCO 4") #changes number of pulses
bnc.write(":PULSE2:WIDT 0.0001")
bnc.write(":PULSE2:DEL 0.031")


19

In [266]:
#DAQ at 0.1Hz for the other devices
bnc.write(":PULSE3:STAT ON")
bnc.write(":PULSE3:CMOD BURST")
bnc.write(":PULSE3:BCO 1") #changes number of pulses
bnc.write(":PULSE3:WIDT 0.0001")
bnc.write(":PULSE3:DEL 0")


15

In [223]:
#pulses in general 
bnc.write(":PULSE2:CMOD BURST")
bnc.write(":PULSE2:BCO 2") #changes number of pulses
bnc.write(":PULSE2:WIDT 0.000625")
#bnc.write(":PULSE1:WIDT 0.0001")
bnc.write(":PULSE2:DEL 0")
bnc.write(":PULSE2:STAT ON")

18

In [224]:
bnc.write(":PULSE2:STAT OFF")
bnc.write(":PULSE3:STAT OFF")

18

In [211]:
#alternative: use 10Mhz external clock to synch BNC555 and DS360
#just generate pulses then from internal clock
bnc.write(":PULSE0:MODE CONT")
bnc.write(":PULSE0:PER 0.00001") #10kHz internal clock
bnc.write(":PULSE0:STAT ON")

17

In [214]:
#now just use the internal clock to get the rest of the frequencies
#get 1kHz run trigger
bnc.write(":PULSE1:STAT ON")
bnc.write(":PULSE1:CMOD DIV")
bnc.write(":PULSE1:DCO 10")
bnc.write(":PULSE1:WIDT 0.00001")
bnc.write(":PULSE1:DEL 0")

15

In [215]:
#get 800Hz DAQ trigger 
#it is 833Hz right now
bnc.write(":PULSE2:STAT ON")
bnc.write(":PULSE2:CMOD DIV")
bnc.write(":PULSE2:DCO 12")
bnc.write(":PULSE2:WIDT 0.00001")
bnc.write(":PULSE2:DEL 0")

15

In [216]:
#get the 1Hz trigger
bnc.write(":PULSE3:STAT ON")
bnc.write(":PULSE3:CMOD DIV")
bnc.write(":PULSE3:DCO 10000")
bnc.write(":PULSE3:WIDT 0.00001")
bnc.write(":PULSE3:DEL 0")

15

In [17]:

print(rm.list_opened_resources())

[]


In [5]:
for resource in rm.list_opened_resources():
    try:
        resource.close()
        print(f"Closed: {resource.resource_name}")
    except Exception as e:
        print(f"Error closing resource: {e}")

Error closing resource: Invalid session handle. The resource might be closed.
